## attempt 8-Final Attempt

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time
import json
import re

def extract_story_paragraphs(driver):
    """Extract ONLY story paragraphs using the specific XPath pattern"""
    
    paragraphs = []
    
    try:
        parent_xpath = '//*[@id="main_content"]/div[1]/div[2]/div[8]/div[2]'
        
        try:
            story_container = driver.find_element(By.XPATH, parent_xpath)
        except:
            story_container = driver.find_element(By.XPATH, '//*[@id="main_content"]//div[contains(@class, "clear")]')
        
        script = """
        var element = arguments[0];
        var textNodes = [];
        
        for (var i = 0; i < element.childNodes.length; i++) {
            var node = element.childNodes[i];
            if (node.nodeType === Node.TEXT_NODE) {
                var text = node.textContent.trim();
                if (text.length > 20) {
                    textNodes.push(text);
                }
            }
        }
        
        return textNodes;
        """
        
        text_nodes = driver.execute_script(script, story_container)
        
        for text in text_nodes:
            if re.search(r'[\u0600-\u06FF]', text):
                text = re.sub(r'\s+', ' ', text).strip()
                if len(text) > 20:
                    paragraphs.append(text)
        
        return paragraphs
    
    except Exception as e:
        print(f"      Error extracting paragraphs: {e}")
        return []

def main():
    driver = webdriver.Chrome()
    
    # Set longer page load timeout
    driver.set_page_load_timeout(30)  # 30 seconds timeout
    
    all_stories = []
    story_links = []
    
    # Step 1: Loop through pages and collect story links
    print("📚 Collecting story links from all pages...\n")
    
    for page_num in range(1, 21):
        if page_num == 1:
            url = "https://www.urdupoint.com/kids/category/moral-stories.html"
        else:
            url = f"https://www.urdupoint.com/kids/category/moral-stories-page{page_num}.html"
        
        print(f"📄 Page {page_num}: {url}")
        
        try:
            driver.get(url)
            time.sleep(3)
            
            story_boxes = driver.find_elements(By.CSS_SELECTOR, 'a.sharp_box')
            
            page_count = 0
            for box in story_boxes:
                story_url = box.get_attribute('href')
                urdu_title = box.find_element(By.CLASS_NAME, 'title_ur').text
                
                if story_url not in [s['url'] for s in story_links]:
                    story_links.append({'title': urdu_title, 'url': story_url})
                    page_count += 1
            
            print(f"   ✅ Found {page_count} stories (Total: {len(story_links)})\n")
        
        except TimeoutException:
            print(f"   ⚠️ Page load timeout - skipping page {page_num}\n")
            continue
        except Exception as e:
            print(f"   ❌ Error: {e}\n")
    
    print("="*70)
    print(f"✅ Total stories collected: {len(story_links)}\n")
    print("="*70 + "\n")
    
    # Step 2: Extract each story with retry logic
    failed_stories = []
    
    for idx, story_info in enumerate(story_links, 1):
        print(f"[{idx}/{len(story_links)}] {story_info['title']}")
        
        # Try up to 3 times for each story
        max_retries = 3
        retry_count = 0
        success = False
        
        while retry_count < max_retries and not success:
            try:
                driver.get(story_info['url'])
                time.sleep(3)  # Increased delay
                
                wait = WebDriverWait(driver, 15)  # Increased wait time
                wait.until(EC.presence_of_element_located((By.ID, "main_content")))
                
                try:
                    title_elem = driver.find_element(By.CLASS_NAME, 'detail_heading')
                    title = title_elem.text.strip()
                except:
                    title = story_info['title']
                
                paragraphs = extract_story_paragraphs(driver)
                
                if not paragraphs:
                    print(f"   ⚠️ No story content extracted")
                    success = True  # Don't retry for empty content
                    break
                
                full_text = '\n\n'.join(paragraphs)
                
                print(f"   ✅ {len(paragraphs)} paragraphs extracted")
                
                story_data = {
                    'title': title,
                    'paragraphs': paragraphs,
                    'full_text': full_text
                }
                
                all_stories.append(story_data)
                
                if paragraphs:
                    preview = paragraphs[0][:80] + "..." if len(paragraphs[0]) > 80 else paragraphs[0]
                    print(f"   📖 Preview: {preview}")
                
                success = True
                
                # Longer delay between stories (be more polite)
                time.sleep(3)
            
            except TimeoutException:
                retry_count += 1
                if retry_count < max_retries:
                    print(f"   ⏳ Timeout - Retry {retry_count}/{max_retries}")
                    time.sleep(5)  # Wait before retry
                else:
                    print(f"   ❌ Timeout after {max_retries} retries - skipping")
                    failed_stories.append(story_info)
            
            except Exception as e:
                retry_count += 1
                if retry_count < max_retries:
                    print(f"   ⏳ Error - Retry {retry_count}/{max_retries}: {e}")
                    time.sleep(5)
                else:
                    print(f"   ❌ Failed after {max_retries} retries: {e}")
                    failed_stories.append(story_info)
        
        # Restart driver every 50 stories to prevent memory issues
        if idx % 50 == 0:
            print(f"\n   🔄 Restarting browser (processed {idx} stories)...\n")
            driver.quit()
            time.sleep(3)
            driver = webdriver.Chrome()
            driver.set_page_load_timeout(30)
    
    # Step 3: Save to JSON
    print("\n" + "="*70)
    print("💾 Saving to JSON...")
    
    output_file = 'urdu_stories_clean.json'
    
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(all_stories, f, ensure_ascii=False, indent=2)
    
    total_paragraphs = sum(len(story['paragraphs']) for story in all_stories)
    total_words = sum(len(story['full_text'].split()) for story in all_stories)
    
    print(f"✅ Saved {len(all_stories)} stories to: {output_file}")
    print(f"📊 Statistics:")
    print(f"   - Total stories: {len(all_stories)}")
    print(f"   - Failed stories: {len(failed_stories)}")
    print(f"   - Total paragraphs: {total_paragraphs}")
    print(f"   - Total words: {total_words:,}")
    if all_stories:
        print(f"   - Avg paragraphs/story: {total_paragraphs/len(all_stories):.1f}")
        print(f"   - Avg words/story: {total_words/len(all_stories):.1f}")
    
    # Save failed stories list
    if failed_stories:
        print(f"\n⚠️ Saving list of failed stories...")
        with open('failed_stories.json', 'w', encoding='utf-8') as f:
            json.dump(failed_stories, f, ensure_ascii=False, indent=2)
        print(f"   Saved to: failed_stories.json")
    
    print("="*70)
    
    driver.quit()

if __name__ == "__main__":
    main()

📚 Collecting story links from all pages...

📄 Page 1: https://www.urdupoint.com/kids/category/moral-stories.html
   ✅ Found 12 stories (Total: 12)

📄 Page 2: https://www.urdupoint.com/kids/category/moral-stories-page2.html
   ✅ Found 12 stories (Total: 24)

📄 Page 3: https://www.urdupoint.com/kids/category/moral-stories-page3.html
   ✅ Found 12 stories (Total: 36)

📄 Page 4: https://www.urdupoint.com/kids/category/moral-stories-page4.html
   ✅ Found 12 stories (Total: 48)

📄 Page 5: https://www.urdupoint.com/kids/category/moral-stories-page5.html
   ⚠️ Page load timeout - skipping page 5

📄 Page 6: https://www.urdupoint.com/kids/category/moral-stories-page6.html
   ✅ Found 12 stories (Total: 60)

📄 Page 7: https://www.urdupoint.com/kids/category/moral-stories-page7.html
   ✅ Found 12 stories (Total: 72)

📄 Page 8: https://www.urdupoint.com/kids/category/moral-stories-page8.html
   ✅ Found 12 stories (Total: 84)

📄 Page 9: https://www.urdupoint.com/kids/category/moral-stories-page9.html

## Preprocessing adding EOP,EOS,EOT

In [3]:
import json
import re
import unicodedata

def full_preprocessing(input_file='urdu_stories_clean.json'):
    """Complete preprocessing pipeline"""
    
    with open(input_file, 'r', encoding='utf-8') as f:
        stories = json.load(f)
    
    print("🔧 Preprocessing Pipeline Started\n")
    
    # Special tokens
    EOS = " <EOS> "
    EOP = " <EOP> "
    EOT = " <EOT> "
    
    processed_stories = []
    
    for idx, story in enumerate(stories, 1):
        print(f"[{idx}/{len(stories)}] {story['title'][:40]}...")
        
        cleaned_paragraphs = []
        
        for paragraph in story['paragraphs']:
            # Step 1: Remove extra whitespace
            text = re.sub(r'\s+', ' ', paragraph).strip()
            
            # Step 2: Normalize Unicode (NFC normalization)
            text = unicodedata.normalize('NFC', text)
            
            # Step 3: Add <EOS> after sentences
            # Split by Urdu sentence endings
            sentences = re.split(r'([۔!?؟])', text)
            
            processed_text = ""
            for i in range(0, len(sentences)-1, 2):
                sentence = sentences[i].strip()
                punct = sentences[i+1] if i+1 < len(sentences) else ""
                if sentence:
                    processed_text += sentence + punct + EOS
            
            # Handle last part without punctuation
            if len(sentences) % 2 == 1 and sentences[-1].strip():
                processed_text += sentences[-1].strip() + EOS
            
            # Step 4: Add <EOP> after paragraph
            processed_text = processed_text.strip() + EOP
            
            cleaned_paragraphs.append(processed_text)
        
        # Step 5: Combine and add <EOT>
        story_text = " ".join(cleaned_paragraphs) + EOT
        
        processed_stories.append({
            'title': story['title'],
            'text': story_text
        })
    
    # Save for BPE tokenizer training
    print(f"\n💾 Saving preprocessed corpus...")
    
    with open('preprocessed_corpus.txt', 'w', encoding='utf-8') as f:
        for story in processed_stories:
            f.write(story['text'] + "\n\n")
    
    # Also save as JSON (optional)
    with open('preprocessed_stories.json', 'w', encoding='utf-8') as f:
        json.dump(processed_stories, f, ensure_ascii=False, indent=2)
    
    print(f"✅ Preprocessed corpus saved!")
    print(f"   Text file: preprocessed_corpus.txt (for tokenizer)")
    print(f"   JSON file: preprocessed_stories.json (backup)")
    
    # Statistics
    full_text = " ".join([s['text'] for s in processed_stories])
    
    print(f"\n📊 Final Statistics:")
    print(f"   Stories: {len(processed_stories)}")
    print(f"   <EOS> count: {full_text.count('<EOS>'):,}")
    print(f"   <EOP> count: {full_text.count('<EOP>'):,}")
    print(f"   <EOT> count: {full_text.count('<EOT>'):,}")
    print(f"   Total chars: {len(full_text):,}")
    
    return processed_stories

if __name__ == "__main__":
    full_preprocessing('urdu_stories_clean.json')

🔧 Preprocessing Pipeline Started

[1/227] پُراسرار بوڑھا...
[2/227] نیا عزم...
[3/227] گھر میں مجرم...
[4/227] جنت کا راستہ...
[5/227] بھوتوں کا نام...
[6/227] ایمانداری کا انعام...
[7/227] رحم کا صلہ...
[8/227] ہرن کی لاپرواہی...
[9/227] ہمت...
[10/227] کرو مہربانی تم اہلِ زمین پر...
[11/227] پانڈا اور اسمارٹ فون...
[12/227] قلم کی خواہش...
[13/227] انمول دوستی...
[14/227] نانی کا سامان...
[15/227] خالی پنجرہ...
[16/227] طاقتور بے وقوف...
[17/227] لکڑ ہارے کا بیٹا...
[18/227] اژدہے کا وعدہ...
[19/227] یہ بکرا کس کا ہے؟...
[20/227] نیکی کا صلہ...
[21/227] لگن...
[22/227] پری کی ممتا...
[23/227] علم کی انتہا جہالت ہے...
[24/227] سپر ہیرو...
[25/227] تیمور کی زندگی...
[26/227] کوا...
[27/227] چھوٹی سی خبر...
[28/227] انعام...
[29/227] نیکی...
[30/227] دوست وہ جو مصیبت میں کام آئے...
[31/227] چھوٹی سی نیکی...
[32/227] دیانت کا صلہ...
[33/227] پرندوں کا پاکستان...
[34/227] برگد کا درخت...
[35/227] علم خزانہ...
[36/227] لالچ کا انجام...
[37/227] کڑوے پتے...
[38/227] اصلی محسن...
[39/227] دو

## BPE Tokenizer

In [4]:
"""
BPE Tokenizer - Saves to JSON
Better for deployment and interoperability
"""

import json
from collections import defaultdict, Counter

class BPETokenizer:
    def __init__(self, vocab_size=250):
        self.vocab_size = vocab_size
        self.vocab = {}
        self.merges = {}
        self.id_to_token = {}
    
    def get_stats(self, words):
        """Count frequency of adjacent byte pairs"""
        pairs = defaultdict(int)
        
        for word, freq in words.items():
            symbols = word.split()
            for i in range(len(symbols) - 1):
                pairs[symbols[i], symbols[i + 1]] += freq
        
        return pairs
    
    def merge_vocab(self, pair, words):
        """Merge the most frequent pair in vocabulary"""
        new_words = {}
        bigram = ' '.join(pair)
        replacement = ''.join(pair)
        
        for word in words:
            new_word = word.replace(bigram, replacement)
            new_words[new_word] = words[word]
        
        return new_words
    
    def train(self, corpus_file):
        """Train BPE tokenizer on corpus"""
        
        print("="*70)
        print("🔤 BPE TOKENIZER TRAINING")
        print("="*70)
        print(f"Target vocabulary size: {self.vocab_size}\n")
        
        # Load corpus
        print("📖 Loading corpus...")
        with open(corpus_file, 'r', encoding='utf-8') as f:
            text = f.read()
        
        print(f"✅ Corpus loaded: {len(text):,} characters\n")
        
        # Build initial vocabulary
        print("🔧 Building character-level vocabulary...")
        
        words = text.split()
        word_freqs = Counter(words)
        
        # Initialize with character sequences
        vocab = {}
        for word, freq in word_freqs.items():
            vocab[' '.join(list(word)) + ' </w>'] = freq
        
        print(f"✅ Initial vocabulary: {len(set(''.join(vocab.keys())))} unique characters\n")
        
        # Get base vocabulary
        base_vocab = set()
        for word in vocab.keys():
            base_vocab.update(word.split())
        
        # Learn merges
        print("🔄 Learning BPE merges...\n")
        
        num_merges = self.vocab_size - len(base_vocab)
        
        for i in range(num_merges):
            pairs = self.get_stats(vocab)
            
            if not pairs:
                break
            
            best_pair = max(pairs, key=pairs.get)
            vocab = self.merge_vocab(best_pair, vocab)
            
            # Store merge as string key for JSON compatibility
            merge_key = f"{best_pair[0]}|{best_pair[1]}"
            self.merges[merge_key] = ''.join(best_pair)
            
            if (i + 1) % 10 == 0:
                print(f"  [{i + 1}/{num_merges}] Merged: {best_pair} (freq: {pairs[best_pair]:,})")
        
        print(f"\n✅ Completed {len(self.merges)} merges\n")
        
        # Build final vocabulary
        print("📚 Building final vocabulary...")
        
        self.vocab = {char: idx for idx, char in enumerate(sorted(base_vocab))}
        
        current_idx = len(self.vocab)
        for merge_key, merged in self.merges.items():
            if merged not in self.vocab:
                self.vocab[merged] = current_idx
                current_idx += 1
        
        self.id_to_token = {idx: token for token, idx in self.vocab.items()}
        
        print(f"✅ Final vocabulary size: {len(self.vocab)}")
        
        return self.vocab
    
    def tokenize(self, text):
        """Tokenize text using learned BPE"""
        
        words = text.split()
        tokens = []
        
        for word in words:
            word_tokens = list(word) + ['</w>']
            
            # Apply merges
            while True:
                pairs = [(word_tokens[i], word_tokens[i + 1]) 
                        for i in range(len(word_tokens) - 1)]
                
                bigram_to_merge = None
                min_merge_idx = float('inf')
                
                for pair in pairs:
                    merge_key = f"{pair[0]}|{pair[1]}"
                    if merge_key in self.merges:
                        merge_order = list(self.merges.keys()).index(merge_key)
                        if merge_order < min_merge_idx:
                            min_merge_idx = merge_order
                            bigram_to_merge = pair
                
                if bigram_to_merge is None:
                    break
                
                first, second = bigram_to_merge
                new_token = first + second
                new_word_tokens = []
                
                i = 0
                while i < len(word_tokens):
                    if i < len(word_tokens) - 1 and \
                       word_tokens[i] == first and word_tokens[i + 1] == second:
                        new_word_tokens.append(new_token)
                        i += 2
                    else:
                        new_word_tokens.append(word_tokens[i])
                        i += 1
                
                word_tokens = new_word_tokens
            
            # Convert to IDs
            for token in word_tokens:
                if token in self.vocab:
                    tokens.append(self.vocab[token])
                else:
                    tokens.append(self.vocab.get('<UNK>', 0))
        
        return tokens
    
    def decode(self, token_ids):
        """Decode token IDs back to text"""
        
        tokens = [self.id_to_token.get(idx, '<UNK>') for idx in token_ids]
        text = ''.join(tokens).replace('</w>', ' ').strip()
        
        return text
    
    def save(self, filepath='bpe_tokenizer.json'):
        """Save tokenizer to JSON file"""
        
        tokenizer_data = {
            'vocab': self.vocab,
            'merges': self.merges,
            'id_to_token': {str(k): v for k, v in self.id_to_token.items()},  # JSON keys must be strings
            'vocab_size': self.vocab_size
        }
        
        with open(filepath, 'w', encoding='utf-8') as f:
            json.dump(tokenizer_data, f, ensure_ascii=False, indent=2)
        
        print(f"✅ Tokenizer saved to: {filepath}")
    
    def load(self, filepath='bpe_tokenizer.json'):
        """Load tokenizer from JSON file"""
        
        with open(filepath, 'r', encoding='utf-8') as f:
            tokenizer_data = json.load(f)
        
        self.vocab = tokenizer_data['vocab']
        self.merges = tokenizer_data['merges']
        self.id_to_token = {int(k): v for k, v in tokenizer_data['id_to_token'].items()}  # Convert back to int
        self.vocab_size = tokenizer_data['vocab_size']
        
        print(f"✅ Tokenizer loaded from: {filepath}")


def main():
    # Train tokenizer
    tokenizer = BPETokenizer(vocab_size=250)
    tokenizer.train('preprocessed_corpus.txt')
    
    # Save as JSON
    tokenizer.save('bpe_tokenizer.json')
    
    # Test
    print("\n" + "="*70)
    print("🧪 Testing Tokenizer")
    print("="*70 + "\n")
    
    test_text = "ایک دن کی بات ہے <EOS>"
    print(f"Original: {test_text}")
    
    tokens = tokenizer.tokenize(test_text)
    print(f"Tokens: {tokens}")
    
    decoded = tokenizer.decode(tokens)
    print(f"Decoded: {decoded}")
    
    print("\n✅ Done!")


if __name__ == "__main__":
    main()

🔤 BPE TOKENIZER TRAINING
Target vocabulary size: 250

📖 Loading corpus...
✅ Corpus loaded: 720,528 characters

🔧 Building character-level vocabulary...
✅ Initial vocabulary: 112 unique characters

🔄 Learning BPE merges...

  [10/139] Merged: ('ں', '</w>') (freq: 10,699)
  [20/139] Merged: ('م', 'یں</w>') (freq: 3,793)
  [30/139] Merged: ('و', 'ں</w>') (freq: 3,032)
  [40/139] Merged: ('ہ', 'و') (freq: 2,296)
  [50/139] Merged: ('ا', 'ل') (freq: 1,811)
  [60/139] Merged: ('بھ', 'ی</w>') (freq: 1,455)
  [70/139] Merged: ('ی', 'ہ</w>') (freq: 1,173)
  [80/139] Merged: ('ج', 'ا') (freq: 968)
  [90/139] Merged: ('خ', 'و') (freq: 875)
  [100/139] Merged: ('ب', 'ہ') (freq: 782)
  [110/139] Merged: (':', '”') (freq: 687)
  [120/139] Merged: ('ب', 'چ') (freq: 623)
  [130/139] Merged: ('تھ', 'ے۔</w>') (freq: 545)

✅ Completed 139 merges

📚 Building final vocabulary...
✅ Final vocabulary size: 250
✅ Tokenizer saved to: bpe_tokenizer.json

🧪 Testing Tokenizer

Original: ایک دن کی بات ہے <EOS>
Toke

In [ ]:
"""
BPE Tokenizer - Loads from JSON
"""

import json


class BPETokenizer:
    def __init__(self):
        self.vocab = {}
        self.merges = {}
        self.id_to_token = {}
        self.vocab_size = 0
    
    def load(self, filepath='bpe_tokenizer.json'):
        """Load tokenizer from JSON file"""
        with open(filepath, 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        self.vocab = data['vocab']
        self.merges = data['merges']
        self.id_to_token = {int(k): v for k, v in data['id_to_token'].items()}
        self.vocab_size = data['vocab_size']
        
        print(f"✅ Tokenizer loaded: vocab size = {len(self.vocab)}")
    
    def tokenize(self, text):
        """Tokenize text into token IDs"""
        words = text.split()
        tokens = []
        
        for word in words:
            word_tokens = list(word) + ['</w>']
            
            # Apply merges
            while True:
                pairs = [(word_tokens[i], word_tokens[i + 1]) 
                        for i in range(len(word_tokens) - 1)]
                
                bigram_to_merge = None
                min_merge_idx = float('inf')
                
                for pair in pairs:
                    merge_key = f"{pair[0]}|{pair[1]}"
                    if merge_key in self.merges:
                        merge_order = list(self.merges.keys()).index(merge_key)
                        if merge_order < min_merge_idx:
                            min_merge_idx = merge_order
                            bigram_to_merge = pair
                
                if bigram_to_merge is None:
                    break
                
                first, second = bigram_to_merge
                new_token = first + second
                new_word_tokens = []
                
                i = 0
                while i < len(word_tokens):
                    if i < len(word_tokens) - 1 and \
                       word_tokens[i] == first and word_tokens[i + 1] == second:
                        new_word_tokens.append(new_token)
                        i += 2
                    else:
                        new_word_tokens.append(word_tokens[i])
                        i += 1
                
                word_tokens = new_word_tokens
            
            # Convert to IDs
            for token in word_tokens:
                if token in self.vocab:
                    tokens.append(self.vocab[token])
                else:
                    tokens.append(self.vocab.get('<UNK>', 0))
        
        return tokens
    
    def decode(self, token_ids):
        """Decode token IDs back to text"""
        tokens = [self.id_to_token.get(idx, '<UNK>') for idx in token_ids]
        text = ''.join(tokens).replace('</w>', ' ').strip()
        return text

## Trigram Language Model

In [5]:
"""
Trigram Language Model - Saves to JSON
"""

import json
import numpy as np
from collections import defaultdict, Counter

class TrigramLanguageModel:
    def __init__(self, lambda1=0.1, lambda2=0.3, lambda3=0.6):
        assert abs(lambda1 + lambda2 + lambda3 - 1.0) < 1e-6
        
        self.lambda1 = lambda1
        self.lambda2 = lambda2
        self.lambda3 = lambda3
        
        self.unigram_counts = Counter()
        self.bigram_counts = defaultdict(Counter)
        self.trigram_counts = defaultdict(Counter)
        
        self.total_tokens = 0
        self.vocab = set()
        
        self.EOS = '<EOS>'
        self.EOP = '<EOP>'
        self.EOT = '<EOT>'
        self.START = '<START>'
    
    def train(self, tokenized_corpus):
        """Train trigram model"""
        
        print("="*70)
        print("📊 TRIGRAM MODEL TRAINING")
        print("="*70)
        print(f"Interpolation: λ1={self.lambda1}, λ2={self.lambda2}, λ3={self.lambda3}\n")
        
        print(f"📖 Training on {len(tokenized_corpus):,} tokens...\n")
        
        # Add start tokens
        tokens = [self.START, self.START] + tokenized_corpus
        
        self.total_tokens = len(tokens)
        self.vocab = set(tokens)
        
        print("🔢 Counting n-grams...")
        
        # Count unigrams
        for token in tokens:
            self.unigram_counts[token] += 1
        
        # Count bigrams
        for i in range(len(tokens) - 1):
            w1 = tokens[i]
            w2 = tokens[i + 1]
            self.bigram_counts[w1][w2] += 1
        
        # Count trigrams
        for i in range(len(tokens) - 2):
            w1 = tokens[i]
            w2 = tokens[i + 1]
            w3 = tokens[i + 2]
            self.trigram_counts[(w1, w2)][w3] += 1
        
        print(f"✅ Unigrams: {len(self.unigram_counts):,}")
        print(f"✅ Bigrams: {sum(len(v) for v in self.bigram_counts.values()):,}")
        print(f"✅ Trigrams: {sum(len(v) for v in self.trigram_counts.values()):,}")
        print("\n✅ Training complete!")
    
    def get_probability(self, w1, w2, w3):
        """Calculate interpolated probability"""
        
        # Unigram
        p_unigram = self.unigram_counts[w3] / self.total_tokens
        
        # Bigram
        if self.unigram_counts[w2] > 0:
            p_bigram = self.bigram_counts[w2][w3] / self.unigram_counts[w2]
        else:
            p_bigram = 0
        
        # Trigram
        if self.bigram_counts[w1][w2] > 0:
            p_trigram = self.trigram_counts[(w1, w2)][w3] / self.bigram_counts[w1][w2]
        else:
            p_trigram = 0
        
        # Interpolated
        p_interpolated = (
            self.lambda1 * p_unigram +
            self.lambda2 * p_bigram +
            self.lambda3 * p_trigram
        )
        
        return p_interpolated
    
    def generate(self, prefix_tokens=None, max_length=500, temperature=1.0):
        """Generate text"""
        
        if prefix_tokens is None or len(prefix_tokens) < 2:
            generated = [self.START, self.START]
        else:
            generated = list(prefix_tokens)
        
        if len(generated) < 2:
            generated = [self.START] + generated
        
        for _ in range(max_length):
            w1 = generated[-2]
            w2 = generated[-1]
            
            candidates = self.vocab
            probabilities = []
            valid_candidates = []
            
            for candidate in candidates:
                prob = self.get_probability(w1, w2, candidate)
                if prob > 0:
                    probabilities.append(prob)
                    valid_candidates.append(candidate)
            
            if not valid_candidates:
                break
            
            # Temperature sampling
            if temperature != 1.0:
                probabilities = np.array(probabilities)
                probabilities = np.power(probabilities, 1.0 / temperature)
                probabilities = probabilities / probabilities.sum()
            else:
                total = sum(probabilities)
                probabilities = [p / total for p in probabilities]
            
            next_token = np.random.choice(valid_candidates, p=probabilities)
            generated.append(next_token)
            
            if next_token == self.EOT:
                break
        
        generated = [t for t in generated if t != self.START]
        
        return generated
    
    def save(self, filepath='trigram_model.json'):
        """Save model to JSON"""
        
        # Convert to JSON-serializable format
        model_data = {
            'lambda1': self.lambda1,
            'lambda2': self.lambda2,
            'lambda3': self.lambda3,
            'unigram_counts': dict(self.unigram_counts),
            'bigram_counts': {str(k): dict(v) for k, v in self.bigram_counts.items()},
            'trigram_counts': {f"{k[0]}|{k[1]}": dict(v) for k, v in self.trigram_counts.items()},
            'total_tokens': self.total_tokens,
            'vocab': list(self.vocab)
        }
        
        with open(filepath, 'w', encoding='utf-8') as f:
            json.dump(model_data, f, ensure_ascii=False, indent=2)
        
        print(f"✅ Model saved to: {filepath}")
    
    def load(self, filepath='trigram_model.json'):
        """Load model from JSON"""
        
        with open(filepath, 'r', encoding='utf-8') as f:
            model_data = json.load(f)
        
        self.lambda1 = model_data['lambda1']
        self.lambda2 = model_data['lambda2']
        self.lambda3 = model_data['lambda3']
        self.unigram_counts = Counter(model_data['unigram_counts'])
        self.bigram_counts = defaultdict(Counter,
                                        {k: Counter(v) for k, v in model_data['bigram_counts'].items()})
        
        # Convert trigram keys back
        trigram_data = {}
        for key, value in model_data['trigram_counts'].items():
            w1, w2 = key.split('|')
            trigram_data[(w1, w2)] = Counter(value)
        self.trigram_counts = defaultdict(Counter, trigram_data)
        
        self.total_tokens = model_data['total_tokens']
        self.vocab = set(model_data['vocab'])
        
        print(f"✅ Model loaded from: {filepath}")


def main():
    # Load tokenizer
    print("🔤 Loading tokenizer...")
    tokenizer = BPETokenizer()
    tokenizer.load('bpe_tokenizer.json')
    
    # Tokenize corpus
    print("\n📖 Loading and tokenizing corpus...")
    with open('preprocessed_corpus.txt', 'r', encoding='utf-8') as f:
        corpus = f.read()
    
    tokenized = tokenizer.tokenize(corpus)
    print(f"✅ Tokenized: {len(tokenized):,} tokens\n")
    
    # Train model
    model = TrigramLanguageModel(lambda1=0.1, lambda2=0.3, lambda3=0.6)
    model.train(tokenized)
    
    # Save as JSON
    model.save('trigram_model.json')
    
    # Test
    print("\n" + "="*70)
    print("🧪 Testing Generation")
    print("="*70 + "\n")
    
    prefix = "ایک دن"
    prefix_tokens = tokenizer.tokenize(prefix)
    
    print(f"Prefix: {prefix}")
    generated = model.generate(prefix_tokens, max_length=50)
    text = tokenizer.decode(generated)
    
    print(f"Generated:\n{text}")
    
    print("\n✅ Done!")


if __name__ == "__main__":
    main()

🔤 Loading tokenizer...
✅ Tokenizer loaded from: bpe_tokenizer.json

📖 Loading and tokenizing corpus...
✅ Tokenized: 353,464 tokens

📊 TRIGRAM MODEL TRAINING
Interpolation: λ1=0.1, λ2=0.3, λ3=0.6

📖 Training on 353,464 tokens...

🔢 Counting n-grams...
✅ Unigrams: 243
✅ Bigrams: 11,777
✅ Trigrams: 85,206

✅ Training complete!
✅ Model saved to: trigram_model.json

🧪 Testing Generation

Prefix: ایک دن
Generated:
ایک دن <UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK>

✅ Done!


## testing bpe and trigram

In [6]:
"""
Test Script - Verify BPE Tokenizer and Trigram Model
This will test if your trained models work correctly
"""

import json
import sys

class BPETokenizer:
    def __init__(self):
        self.vocab = {}
        self.merges = {}
        self.id_to_token = {}
        self.vocab_size = 0
    
    def load(self, filepath='bpe_tokenizer.json'):
        """Load tokenizer from JSON"""
        with open(filepath, 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        self.vocab = data['vocab']
        self.merges = data['merges']
        self.id_to_token = {int(k): v for k, v in data['id_to_token'].items()}
        self.vocab_size = data['vocab_size']
        
        print(f"✅ Tokenizer loaded")
        print(f"   Vocabulary size: {len(self.vocab)}")
        print(f"   Merges learned: {len(self.merges)}")
    
    def tokenize(self, text):
        """Tokenize text"""
        words = text.split()
        tokens = []
        
        for word in words:
            word_tokens = list(word) + ['</w>']
            
            # Apply merges
            while True:
                pairs = [(word_tokens[i], word_tokens[i + 1]) 
                        for i in range(len(word_tokens) - 1)]
                
                bigram_to_merge = None
                min_merge_idx = float('inf')
                
                for pair in pairs:
                    merge_key = f"{pair[0]}|{pair[1]}"
                    if merge_key in self.merges:
                        merge_order = list(self.merges.keys()).index(merge_key)
                        if merge_order < min_merge_idx:
                            min_merge_idx = merge_order
                            bigram_to_merge = pair
                
                if bigram_to_merge is None:
                    break
                
                first, second = bigram_to_merge
                new_token = first + second
                new_word_tokens = []
                
                i = 0
                while i < len(word_tokens):
                    if i < len(word_tokens) - 1 and \
                       word_tokens[i] == first and word_tokens[i + 1] == second:
                        new_word_tokens.append(new_token)
                        i += 2
                    else:
                        new_word_tokens.append(word_tokens[i])
                        i += 1
                
                word_tokens = new_word_tokens
            
            # Convert to IDs
            for token in word_tokens:
                if token in self.vocab:
                    tokens.append(self.vocab[token])
                else:
                    # Unknown token
                    tokens.append(self.vocab.get('<UNK>', 0))
        
        return tokens
    
    def decode(self, token_ids):
        """Decode token IDs back to text"""
        tokens = [self.id_to_token.get(idx, '<UNK>') for idx in token_ids]
        text = ''.join(tokens).replace('</w>', ' ').strip()
        return text


def test_tokenizer():
    """Test the BPE tokenizer"""
    
    print("\n" + "="*70)
    print("🧪 TESTING BPE TOKENIZER")
    print("="*70)
    
    # Load tokenizer
    tokenizer = BPETokenizer()
    tokenizer.load('bpe_tokenizer.json')
    
    # Test 1: Basic tokenization
    print("\n📝 Test 1: Basic Tokenization")
    test_text = "ایک دن کی بات ہے"
    
    print(f"   Input: {test_text}")
    
    tokens = tokenizer.tokenize(test_text)
    print(f"   Token IDs: {tokens}")
    print(f"   Number of tokens: {len(tokens)}")
    
    decoded = tokenizer.decode(tokens)
    print(f"   Decoded: {decoded}")
    
    if test_text == decoded:
        print("   ✅ PASS: Decoding matches original")
    else:
        print("   ⚠️ WARN: Decoding differs slightly (normal for some tokens)")
    
    # Test 2: Special tokens
    print("\n📝 Test 2: Special Tokens")
    test_with_special = "کہانی ختم ہوئی <EOS> <EOP> <EOT>"
    
    print(f"   Input: {test_with_special}")
    
    tokens = tokenizer.tokenize(test_with_special)
    print(f"   Token IDs: {tokens}")
    
    decoded = tokenizer.decode(tokens)
    print(f"   Decoded: {decoded}")
    
    if "<EOS>" in decoded or "<EOP>" in decoded or "<EOT>" in decoded:
        print("   ✅ PASS: Special tokens preserved")
    else:
        print("   ⚠️ WARN: Special tokens not fully preserved")
    
    # Test 3: Vocabulary coverage
    print("\n📝 Test 3: Vocabulary Coverage")
    
    # Check for important Urdu characters
    urdu_chars = ['ا', 'ب', 'ت', 'ک', 'م', 'ن', 'ی', 'ے']
    found = sum(1 for char in urdu_chars if char in tokenizer.vocab)
    
    print(f"   Urdu characters in vocab: {found}/{len(urdu_chars)}")
    
    # Check for special tokens
    special_tokens = ['<EOS>', '<EOP>', '<EOT>']
    has_special = any(token in str(tokenizer.vocab.keys()) for token in special_tokens)
    
    if has_special:
        print(f"   ✅ Special tokens found in vocabulary")
    else:
        print(f"   ⚠️ Special tokens not in vocabulary")
    
    print("\n✅ Tokenizer tests complete!")
    
    return tokenizer


def test_trigram_model():
    """Test the trigram model"""
    
    print("\n" + "="*70)
    print("🧪 TESTING TRIGRAM MODEL")
    print("="*70)
    
    # Load model
    print("\n📖 Loading trigram model...")
    
    with open('trigram_model.json', 'r', encoding='utf-8') as f:
        model_data = json.load(f)
    
    print(f"✅ Model loaded")
    print(f"   Lambda values: λ1={model_data['lambda1']}, λ2={model_data['lambda2']}, λ3={model_data['lambda3']}")
    
    # Check unigrams
    unigram_count = len(model_data['unigram_counts'])
    print(f"   Unigram tokens: {unigram_count:,}")
    
    # Check bigrams
    bigram_count = len(model_data['bigram_counts'])
    print(f"   Bigram contexts: {bigram_count:,}")
    
    # Check trigrams
    trigram_count = len(model_data['trigram_counts'])
    print(f"   Trigram contexts: {trigram_count:,}")
    
    # Check total tokens
    total_tokens = model_data['total_tokens']
    print(f"   Total tokens trained on: {total_tokens:,}")
    
    # Verify lambdas sum to 1
    lambda_sum = model_data['lambda1'] + model_data['lambda2'] + model_data['lambda3']
    
    if abs(lambda_sum - 1.0) < 0.001:
        print(f"   ✅ Lambda values sum to 1.0")
    else:
        print(f"   ❌ ERROR: Lambda values sum to {lambda_sum}")
    
    print("\n✅ Trigram model tests complete!")


def verify_corpus():
    """Verify the preprocessed corpus"""
    
    print("\n" + "="*70)
    print("🧪 TESTING PREPROCESSED CORPUS")
    print("="*70)
    
    with open('preprocessed_corpus.txt', 'r', encoding='utf-8') as f:
        corpus = f.read()
    
    # Count special tokens
    eos_count = corpus.count('<EOS>')
    eop_count = corpus.count('<EOP>')
    eot_count = corpus.count('<EOT>')
    
    print(f"\n📊 Special Token Counts:")
    print(f"   <EOS> (End of Sentence): {eos_count:,}")
    print(f"   <EOP> (End of Paragraph): {eop_count:,}")
    print(f"   <EOT> (End of Text/Story): {eot_count:,}")
    
    # Estimate number of stories
    print(f"\n📚 Estimated stories in corpus: {eot_count}")
    
    # Check Urdu content
    import re
    urdu_chars = len(re.findall(r'[\u0600-\u06FF]', corpus))
    total_chars = len(corpus)
    
    urdu_percentage = (urdu_chars / total_chars) * 100
    
    print(f"\n📝 Content Analysis:")
    print(f"   Total characters: {total_chars:,}")
    print(f"   Urdu characters: {urdu_chars:,}")
    print(f"   Urdu content: {urdu_percentage:.1f}%")
    
    if urdu_percentage > 70:
        print(f"   ✅ Good Urdu content coverage")
    else:
        print(f"   ⚠️ Low Urdu content (might have too many English characters)")
    
    print("\n✅ Corpus verification complete!")


def main():
    """Run all tests"""
    
    print("="*70)
    print("🎯 COMPLETE MODEL VERIFICATION")
    print("="*70)
    
    try:
        # Test 1: Corpus
        verify_corpus()
        
        # Test 2: Tokenizer
        tokenizer = test_tokenizer()
        
        # Test 3: Trigram Model
        test_trigram_model()
        
        # Final summary
        print("\n" + "="*70)
        print("✅ ALL TESTS PASSED!")
        print("="*70)
        print("\n📋 Summary:")
        print("   ✅ Preprocessed corpus has special tokens")
        print("   ✅ BPE tokenizer trained (vocab size: 250)")
        print("   ✅ Trigram model trained with interpolation")
        print("\n🎉 Your models are ready for deployment!")
        print("\n📝 Next steps:")
        print("   1. Create FastAPI backend (Phase IV)")
        print("   2. Build web frontend (Phase V)")
        print("   3. Deploy to Vercel (Phase VI)")
        
    except FileNotFoundError as e:
        print(f"\n❌ ERROR: File not found - {e}")
        print("Make sure these files are in the same directory:")
        print("   - bpe_tokenizer.json")
        print("   - trigram_model.json")
        print("   - preprocessed_corpus.txt")
    
    except Exception as e:
        print(f"\n❌ ERROR: {e}")
        import traceback
        traceback.print_exc()


if __name__ == "__main__":
    main()


🎯 COMPLETE MODEL VERIFICATION

🧪 TESTING PREPROCESSED CORPUS

📊 Special Token Counts:
   <EOS> (End of Sentence): 10,810
   <EOP> (End of Paragraph): 3,396
   <EOT> (End of Text/Story): 227

📚 Estimated stories in corpus: 227

📝 Content Analysis:
   Total characters: 720,528
   Urdu characters: 481,695
   Urdu content: 66.9%
   ⚠️ Low Urdu content (might have too many English characters)

✅ Corpus verification complete!

🧪 TESTING BPE TOKENIZER
✅ Tokenizer loaded
   Vocabulary size: 250
   Merges learned: 139

📝 Test 1: Basic Tokenization
   Input: ایک دن کی بات ہے
   Token IDs: [166, 66, 151, 137, 174, 131, 222]
   Number of tokens: 7
   Decoded: ایک دن کی بات ہے
   ✅ PASS: Decoding matches original

📝 Test 2: Special Tokens
   Input: کہانی ختم ہوئی <EOS> <EOP> <EOT>
   Token IDs: [97, 101, 126, 112, 65, 243, 150, 172, 119, 136, 114, 29, 115]
   Decoded: کہانی ختم ہوئی <EOS> <EOP> <EOT>
   ✅ PASS: Special tokens preserved

📝 Test 3: Vocabulary Coverage
   Urdu characters in vocab: 8/8